In [17]:
import pandas as pd
import itertools
from sklearn.neighbors import KNeighborsRegressor, NearestNeighbors
from dateutil.relativedelta import relativedelta
from sklearn.metrics import mean_absolute_error
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
%matplotlib inline

In [2]:
data = pd.read_csv('/dsg/demand_anonymized_20170802.csv', delimiter=';', parse_dates=["Month"])
eval = pd.read_csv('/dsg/eval.csv')

In [3]:
# create aggregated by month series
series = data.groupby(["SalOrg", "Material", "Month"])["OrderQty"].sum().reset_index()
series = series.sort_values(by=["SalOrg", "Material", "Month"])
series.head()

,SalOrg,Material,Month,OrderQty
0,97LK,00IYcj,2012-05-01,2
1,97LK,00IYcj,2012-06-01,13
2,97LK,00IYcj,2012-07-01,1
3,97LK,00IYcj,2012-09-01,30
4,97LK,00IYcj,2012-11-01,1


In [4]:
# fill empty
eval_comb = eval[['Material', 'SalOrg']]
eval_comb = list(set([tuple(x) for x in eval_comb.values]))

comb = list(itertools.product(*[eval_comb, list(series['Month'].unique())]))
comb = [(t[0], t[1], m) for t, m in comb]

series2 = pd.DataFrame(comb, columns=['Material', 'SalOrg', 'Month'])
series2 = series2.sort_values(by=['Material', 'SalOrg', 'Month' ])
series2 = series2.merge(series, on=['Month', 'Material', 'SalOrg'], how='left')
series2 = series2.fillna(0)

In [5]:
series2['OrderQtyLog'] = np.log1p(series2['OrderQty'])

In [6]:
# create lagged features
def lag_feature(df, colname, lag, adv):
    cols_lagged = []
    for i in range(lag, 0, -1):
        col_lagged = '{}(t-{})'.format(colname, i)
        df[col_lagged] = df.groupby(by=['SalOrg', 'Material'])[colname].shift(i)
        cols_lagged.append(col_lagged)
    cols_adv = []
    for i in range(1, adv):
        col_adv = '{}(t+{})'.format(colname, i)
        df[col_adv] = df.groupby(by=['SalOrg', 'Material'])[colname].shift(-i)
        cols_adv.append(col_adv) 
        
    return df, cols_lagged, cols_adv

series, lagged, adv = lag_feature(series2, 'OrderQtyLog', 12, 3)
series.head()

,Material,SalOrg,Month,OrderQty,OrderQtyLog,OrderQtyLog(t-12),OrderQtyLog(t-11),OrderQtyLog(t-10),OrderQtyLog(t-9),OrderQtyLog(t-8),OrderQtyLog(t-7),OrderQtyLog(t-6),OrderQtyLog(t-5),OrderQtyLog(t-4),OrderQtyLog(t-3),OrderQtyLog(t-2),OrderQtyLog(t-1),OrderQtyLog(t+1),OrderQtyLog(t+2)
0,00GB1f,yqSu,2012-01-01,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.693147
1,00GB1f,yqSu,2012-02-01,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.693147,1.386294
2,00GB1f,yqSu,2012-03-01,1.0,0.693147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.386294,1.098612
3,00GB1f,yqSu,2012-04-01,3.0,1.386294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,0.693147,1.098612,0.693147
4,00GB1f,yqSu,2012-05-01,2.0,1.098612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.693147,1.386294,0.693147,1.386294


In [7]:
# validation folds
def create_validation(data, date, m):
    return data[(data["Month"] > pd.to_datetime(date) - relativedelta(months=m))&
                (data["Month"] < pd.to_datetime(date))].index, data[(data["Month"] >= pd.to_datetime(date)) & 
                (data["Month"] < pd.to_datetime(date) + relativedelta(months=3))].index

def create_agg_features(data, indexes, lags, columns, name):
    temp = data.loc[(data["Month"] > data.loc[indexes, "Month"].min() - relativedelta(years=lags))&
                    (data["Month"] < data.loc[indexes, "Month"].min())].groupby(columns)["OrderQty"].agg(["median", "mean", "std", "min", "max"]).reset_index()
    temp.columns = columns + [col + name + str(lags) for col in ["median", "mean", "std", "min", "max"]]
    return temp

validation_months = ['2016-08-01', '2016-09-01', '2016-10-01', '2016-11-01']

folds = []
for month in validation_months:
    fold = create_validation(series, month, 3)
    folds.append(fold)
    
    temp = series.loc[folds[0]].merge(create_agg_features(series, folds[0], 1, ["SalOrg", "Material"], "_s_m_"), how='left', on=["SalOrg", "Material"])
    temp = temp.merge(create_agg_features(series, folds[0], 1, ["SalOrg"], "_s_"), how='left', on=["SalOrg"])
    temp = temp.merge(create_agg_features(series, folds[0], 1, ["Material"], "_m_"), how='left', on=["Material"])
    series.loc[folds[0]] = temp
    

In [ ]:
# knn model
param_dist = {"n_neighbors": [8, 10],
#               "algorithm": ['ball_tree', 'kd_tree', 'brute'],
              "metric": ['minkowski', 'euclidean', 'canberra']}

n_iter_search = 6
clf = KNeighborsRegressor(n_jobs=8)
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=folds, 
                                   scoring='neg_mean_absolute_error', n_jobs=3,
                                   verbose=1)

X = series[lagged]
y = series[['OrderQtyLog'] + adv]

random_search.fit(X, y)

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            
report(random_search.cv_results_)

# all_errors = []
# for fold in folds:   
#     train = series.loc[fold[0]]
#     test = series.loc[fold[1]]
    
#     train_x = train[lagged]
#     train_y = train[]
#     test_x = test[lagged]
#     test_y = test[['OrderQtyLog'] + adv]
    
#     model = KNeighborsRegressor(n_neighbors=nn, metric='canberra', n_jobs=-1).fit(train_x, train_y)
#     test_y_predicted = model.predict(test_x)
    
#     error = mean_absolute_error(np.expm1(test_y), np.expm1(test_y_predicted))
#     print('Fold error: {}'.format(error))
    
#     all_errors.append(error)
    
# np.mean(all_errors)

Fitting 4 folds for each of 6 candidates, totalling 24 fits


In [9]:
# all dataset 
train_x = test[lagged]
train_y = test[['OrderQtyLog'] + adv]
model = KNeighborsRegressor(n_neighbors=nn, metric='canberra', n_jobs=-1).fit(train_x, train_y)

In [10]:
lagged_final = lagged
lagged_final.append('OrderQtyLog')
lagged_final.pop(0)
lagged_final

['OrderQtyLog(t-11)',
 'OrderQtyLog(t-10)',
 'OrderQtyLog(t-9)',
 'OrderQtyLog(t-8)',
 'OrderQtyLog(t-7)',
 'OrderQtyLog(t-6)',
 'OrderQtyLog(t-5)',
 'OrderQtyLog(t-4)',
 'OrderQtyLog(t-3)',
 'OrderQtyLog(t-2)',
 'OrderQtyLog(t-1)',
 'OrderQtyLog']

In [11]:
test_i = series[series["Month"]==pd.to_datetime('2017-03-01')].index

test_x = series.loc[test_i][lagged_final]
test_prediction = model.predict(test_x)
prediction = np.expm1(test_prediction)
prediction

array([[ 0.        ,  0.        ,  0.25992105],
       [ 2.47602664,  7.17330203,  7.10283902],
       [ 1.62074139,  1.41014226,  3.02072576],
       ..., 
       [ 1.5198421 ,  0.58740105,  0.25992105],
       [ 3.57885697,  2.41995189,  1.5198421 ],
       [ 0.70997595,  0.        ,  0.58740105]])

In [12]:
tst = series.loc[test_i].copy()
tst['OrderQty(t+1)'] = prediction.T[0]
tst['OrderQty(t+2)'] = prediction.T[1]
tst['OrderQty(t+3)'] = prediction.T[2]

In [13]:
ml = pd.melt(tst, id_vars=['Material', 'SalOrg'], value_vars=['OrderQty(t+1)', 'OrderQty(t+2)','OrderQty(t+3)'])

In [14]:
ml['date'] = ml.variable.replace({'OrderQty(t+1)':'2017-04', 'OrderQty(t+2)':'2017-05', 'OrderQty(t+3)':'2017-06'})
ml.head()

,Material,SalOrg,value,date
0,00GB1f,yqSu,0.000000,2017-04
1,00IYcj,97LK,2.476027,2017-04
2,00IYcj,OQfZ,1.620741,2017-04
3,00IYcj,U12J,25.832988,2017-04
4,00IYcj,yqSu,14.951677,2017-04


In [15]:
result = eval.merge(ml, on=['Material', 'SalOrg', 'date'])
result

,ID,SalOrg,Material,date,value
0,0,97LK,00IYcj,2017-04,2.476027
1,1,97LK,00lqzT,2017-04,4.013298
2,2,97LK,00MFcK,2017-04,0.000000
3,3,97LK,00mt9e,2017-04,1.620741
4,4,97LK,00Ok8y,2017-04,0.912931
5,5,97LK,00W03x,2017-04,0.442250
6,6,97LK,02jRc8,2017-04,0.000000
7,7,97LK,02QhQT,2017-04,0.000000
8,8,97LK,0355Np,2017-04,0.817121
9,9,97LK,03eCyI,2017-04,321.008204


In [ ]:
result['demand'] = result['value']
result[['ID', 'demand']].to_csv('knn6.csv', index=False)